## Participants questionnaire data 
at location  /opt/data/optimax_sensing/Klinische\ Variabeln_Gesamt_220120_LA.sav


from columns:
- OASIS_PRE	
- BAI_PRE
- BAI_POST2
- OASIS_POST2
- BAI_POST
- OASIS_POST
- BAI_MID2
- OASIS_MID2
- BAI_MID
- OASIS_MID 




In [5]:
import pandas as pd
import datetime
import pickle

QU_PRE = "/opt/data/optimax_sensing/unipark/PRE_Klinische Variabeln_Gesamt_220120_LA.csv"
QU_MIDa = "/opt/data/optimax_sensing/unipark/MID_Gesamtfile_211213_LA.csv"
QU_MIDb = "/opt/data/optimax_sensing/unipark/MID2_Gesamt_111021_AE.csv"
QU_POSTa = "/opt/data/optimax_sensing/unipark/Post_Gesamt_20220214_LA.csv"
QU_POSTb = "/opt/data/optimax_sensing/unipark/POST2_Gesamt_291021_AE.csv"

def create_df(path): 
    df = pd.read_csv(path,sep = ';')
    return df


In [6]:
df = create_df(QU_PRE)

In [13]:
import re 
parta = ['p102', 'p104', 'p124', 'p131', 'p134', 'p141', 'p149', 'p154', 'p166', 'p170', 'p181', 'p194', 'p201', 'p208', 'p230', 'p255', 'p261', 'p262', 'p263', 'p265', 'p273', 'p284', 'p310', 'p319', 'p326', 'p333', 'p355', 'p366', 'p369', 'p376', 'p383', 'p393', 'p400']
parta = set([int(i[1:]) for i in parta]) #ones in ps dataset 

missing = parta - set([int(re.findall(r'[0-9]+', i, flags=0)[0]) for i in df['VP']]) 

In [12]:

print(set([int(re.findall(r'[0-9]+', i, flags=0)[0]) for i in df['VP']]))

{101, 102, 104, 105, 106, 107, 109, 113, 114, 115, 116, 117, 118, 121, 123, 124, 128, 129, 130, 131, 133, 134, 135, 136, 137, 140, 141, 142, 146, 147, 149, 150, 151, 152, 154, 155, 156, 157, 158, 160, 161, 162, 164, 166, 168, 169, 170, 171, 173, 174, 175, 177, 178, 179, 181, 182, 184, 186, 188, 189, 190, 194, 195, 196, 197, 198, 199, 200, 201, 204, 207, 208, 210, 213, 214, 215, 216, 217, 218, 219, 220, 226, 227, 229, 230, 233, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 253, 255, 257, 258, 261, 262, 263, 264, 265, 266, 267, 268, 269, 271, 273, 275, 277, 279, 282, 283, 284, 285, 286, 288, 290, 291, 292, 294, 295, 296, 297, 298, 299, 302, 310, 311, 312, 313, 315, 316, 317, 319, 323, 325, 326, 333, 335, 336, 340, 344, 345, 346, 348, 349, 350, 351, 352, 354, 355, 366, 369, 370, 371, 372, 374, 376, 382, 383, 387, 392, 393, 394, 396, 400, 401, 404, 405, 407, 408, 409, 411, 413, 415, 416, 417, 418, 421, 423, 424, 425, 427, 428, 429, 430, 431, 433, 434, 439, 441, 443,

## Participants demographic data 
at location  /opt/data/optimax_sensing/Klinische\ Variabeln_Gesamt_220120_LA.sav





In [ ]:
# https://github.com/Roche/pyreadstat




## Participants EMA data 

In [ ]:
import pandas as pd
import datetime
import pickle

GESAMTFILE_b = '/opt/data/optimax_sensing/Gesamtfile_EMA_Baseline_bereinigt_29.01.22_MD.xlsx'
GESAMTFILE_m = '/opt/data/optimax_sensing/Gesamtfile_EMA_MID_29.01.22_MD.xlsx'
GESAMTFILE_p = '/opt/data/optimax_sensing/Gesamtfile_EMA_POST_29.01.22_MD.xlsx'

def create_df(path):
    df = pd.read_excel(path, engine='openpyxl')
    df = df[['VP','Date_Time_last_action (MAX) / completed (SEMA3)','Teilnehmer Mattermost']]

    #Generate placeholder values for missing columns 
    df[['fitbit_id','empatica_id']] = None
    df['platform'] = 'infer'
    df['label'] = df['VP']
    df['end_date'] = None #df['Date_Time_last_action (MAX) / completed (SEMA3)']

    #rename and order columns
    df = df[['Teilnehmer Mattermost','fitbit_id','empatica_id','VP','platform','label','Date_Time_last_action (MAX) / completed (SEMA3)','end_date']]
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']

    #drop NAs in pid column 
    df = df.dropna(subset=['pid'])
    df = df.dropna(subset=['device_id'])

    #Convert dtypes
    df = df.astype({"label": int, "pid": int})
    df['start_date'] = pd.to_datetime(df['start_date'], infer_datetime_format = True)
    #df['start_date'] = df['start_date'].dt.round('D') #round to day, yields inaccurate results 


    #group by participant and get first and last date, put in last two columns 
    end_dates = df.groupby('pid')['start_date'].max().to_frame()
    start_dates = df.groupby('pid')['start_date'].min().to_frame()


    df = df.drop_duplicates(subset=['pid']) #delete duplicate rows 

    df = df.drop(['start_date','end_date'], axis = 1)

    df = df.merge(start_dates, how = 'inner', on = ['pid'])
    df = df.merge(end_dates, how = 'inner', on = ['pid'])

    #rename columns
    df.columns = ['device_id','fitbit_id','empatica_id','pid','platform','label','start_date','end_date']
    return df

